# Text generation models

One very basic kind of text generation model is the Markov model.  In
such a model, we have a state which consists of the previous character.
We also have a matrix of transitions from one character to another.  We
*train* the model by feeding it some text, and observing the
transitions.  We can then generate more text from the model.

In [7]:
from collections import defaultdict
import re

def train_sentence(transitions, sentence, depth=1):
    # We need a "special" character to represent the beginning of a sentence.
    # This is also the character we'll use to feed the generator, below.
    prevchar = "•" * depth
    for char in sentence:
        transitions[prevchar][char] += 1
        prevchar = prevchar[1:] + char

    return transitions

def split_text(text):
    for sentence in re.finditer(".*?([.?!][”’]?|\n\n)", text, re.DOTALL):
        # Turn all sequences of whitespace into a single space
        sentence = re.sub("[ \t\n\r]+", " ", sentence.group(0)).strip()
        if sentence != '':
            yield sentence

def train(filename, depth=1):
    transitions = defaultdict(lambda: defaultdict(int))
    with open(filename) as fin:
        text = fin.read()
        for sentence in split_text(text):
            if len(sentence) < 3:
                continue
            transitions = train_sentence(transitions, sentence, depth)

    return transitions

In [8]:
import pandas as pd
import numpy as np

def format_transitions(trs):
    rows = []
    for key in trs:
        for key2 in trs[key]:
            rows.append({'from': key, 'to': key2, 'n': trs[key][key2]})
    data = pd.DataFrame(rows)
    data = data.pivot_table(index='from', columns='to', values='n')
    data = data.div(data.sum(axis=1), axis=0)
    data[np.isnan(data)] = 0

    return data

In [3]:
def produce(transitions):
    if isinstance(transitions, defaultdict):
        transitions = format_transitions(transitions)

    # Nifty trick: auto-calculate the depth we were trained on
    depth = len(transitions.index[0])

    output = ""
    last = "•" * depth
    nxt = ""

    while nxt not in [".", "?", "!"]:
        trs = transitions.loc[last]
        nxt = np.random.choice(trs.index, p=trs)
        last = last[1:] + nxt
        output += nxt

    return output

In [4]:
tr = train("alice.txt", 4)

In [5]:
[produce(tr) for x in range(5)]

['Supposed up and ther is,’ thoughtfull over unpleast in accur: but states’ll trying--‘Catch the elbow what?',
 'Down,’ said that promoting riddle into them, and the rose things, we should not be folding then!',
 'How CAN I haven’t makes also, and put once, when sat she talk.',
 'This cut it see as shrink I must really far don’t might-eyed the Project Gutenberg License was sense, which was herself.',
 '‘No,’ said Alice if no idea,’ ther.']

Ideas for extension:

- Train on a different text
- Try normalizing the text in different ways (e.g. what happens if you take
  out quotation marks?)
- Play around with different depths.  Do different ones work better for
  different texts?
- Try generating longer passages (you will need to alter the training
  also)
- Convert to word-based rather than character based

# Neural networks

Neural networks in the news

- [An excellent introduction to RNNs and character level language models](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [Generating Harry Potter text with a RNN (with tutorial)](https://medium.com/deep-writing/harry-potter-written-by-artificial-intelligence-8a9431803da6)
  - “Yeah, I know, I saw him run off the balls of the Three Broomsticks around the Daily Prophet that we met Potter’s name!” said Hermione. “We’ve done all right, Draco, and Karkaroff would have to spell the Imperius Curse,” said Dumbledore. “But Harry, never found out about the happy against the school.”
- [The AI that's "too dangerous to release"](https://openai.com/blog/better-language-models/) (NB this specific claim is propaganda, and the model probably does not actually work as it's claimed to)
  - *John F. Kennedy was just elected President of the United States after rising from the grave decades after his assassination. Due to miraculous developments in nanotechnology, Kennedy’s brain was rebuilt from his remains and installed in the control center of a state-of-the art humanoid robot. Below is a transcript of his acceptance speech.*
  
    “It is time once again. I believe this nation can do great things if the people make their voices heard. The men and women of America must once more summon our best elements, all our ingenuity, and find a way to turn such overwhelming tragedy into the opportunity for a greater good and the fulfillment of all our dreams. In the months and years to come, there will be many battles in which we will have to be strong and we must give all of our energy, not to repel invaders, but rather to resist aggression and to win the freedom and the equality for all of our people. The destiny of the human race hangs in the balance; we cannot afford for it to slip away. Now and in the years to come, the challenge before us is to work out how we achieve our ultimate destiny. If we fail to do so, we are doomed.”
    
    [...]

    After the election, the President of the United States asked for a “more complete” body of information on Kennedy’s brains’ origin. In an e-mail exchange with me, Dr. Robert Wood, who, according to Wikipedia is the head of the University of Maryland Advanced Nanotechnology Research project, provided the following information:

    (1) The brain of JFK was harvested and reconstructed via tissue sampling. There was no way that the tissue could be transported by air. (2) A sample was collected from the area around his upper chest and sent to the University of Maryland for analysis.
    
Tutorials:

- [Tensorflow text generation (Shakespeare)](https://www.tensorflow.org/beta/tutorials/text/text_generation)
- [PyTorch name generation/classification](https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html) -- a different task, but with useful conceptual background

Code:
- [Keras](https://github.com/ekzhang/char-rnn-keras) -- on which some of the below code is based

For our neural network, we have a series of layers that do different things:
- Embedding: turn a single number (character-number) into a vector of numbers
- LSTM (long short term memory): a very general kind of neural network gate.  Intuitively, it consists of a single "cell" of memory, with three controls:
  - input: how likely this cell is to "learn" something new
  - forget: how likely this cell is to hold on to what it already knows
  - output: how likely this cell is to pass what it knows on to the rest of the network
- Dropout: a layer that randomly zeroes out part of its contents.  While this might seem like a bad thing, it turns out to prevent overfitting in practice
- TimeDistributed: somewhat misleading name (in this application).  Since we are feeding multiple slices in at once, and predicting multiple outputs, we need this layer to glue the inputs and outputs together.  The multiple characters are the "time" in the name.  ([explanation](https://github.com/keras-team/keras/issues/1029#issuecomment-158105579))
- Activation: transforms to probabilities/weights per character

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, TimeDistributed, Embedding

EMBEDDING_SZ = 128
LSTM_SZ = 64

def build_model(simultaneous_batches, chars_per_batch, vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SZ, batch_input_shape=(simultaneous_batches, chars_per_batch)))
    for i in range(3):
        model.add(LSTM(LSTM_SZ, return_sequences=True, stateful=True))
        model.add(Dropout(0.2))
        
    model.add(TimeDistributed(Dense(vocab_size)))
    model.add(Activation('softmax'))
    
    return model

Using TensorFlow backend.


We want to feed the text into the network in simultaneous batches.  Imagine that we want 4 batches, each 8 characters long.  Imagine too that the text is 96 characters long.  For convenience we'll split the text into 4 lines, and use the 24 letters a-x to represent the text.  The first batch will consist of 4 copies of the string "abcdefgh" (characters 0-7, 24-31, 48-55, and 72-79).  The second batch will consist of 4 copies of "ijklmnop" (8-15, 32-39, 56-63, 80-87).  And the third will consist of 4 copies of "qrstuvwx" (16-23, 40-47, 64-71, 88-95).  Thus, each character is fed into the network exactly once.  The advantage of this scheme is that it spreads any local variation in the text across different training periods, rather than combining it into one.  (Remember: a real text is more likely to be like "aaabbbccc" than "abcabcabc")

```
abcdefghijklmnopqrstuvwx
abcdefghijklmnopqrstuvwx
abcdefghijklmnopqrstuvwx
abcdefghijklmnopqrstuvwx
```

In [3]:
CHARS_PER_BATCH = 64
SIMULTANEOUS_BATCHES = 16
def batches(text):
    alphabet = sorted(set(list(text)))
    distance_between_batches = len(text) // SIMULTANEOUS_BATCHES
    for start in range(0, distance_between_batches - CHARS_PER_BATCH, CHARS_PER_BATCH):
        x = np.zeros((SIMULTANEOUS_BATCHES, CHARS_PER_BATCH))
        y = np.zeros((SIMULTANEOUS_BATCHES, CHARS_PER_BATCH, len(alphabet)))
        for i in range(0, SIMULTANEOUS_BATCHES):
            for j in range(0, CHARS_PER_BATCH):
                x[i, j] = alphabet.index(text[distance_between_batches * i + start + j])
                y[i, j, alphabet.index(text[distance_between_batches * i + start + j + 1])] = 1
        yield x, y

In [4]:
with open("alice.txt") as fin:
    text = fin.read()

In [5]:
alphabet = sorted(list(set(text)))
vocab_size = len(alphabet)
model = build_model(simultaneous_batches=SIMULTANEOUS_BATCHES,
                    chars_per_batch=CHARS_PER_BATCH,
                    vocab_size=vocab_size)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (16, 64, 128)             9088      
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 64)              49408     
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 64)              0         
_________________________________________________________________
lstm_2 (LSTM)                (16, 64, 64)              33024     
_________________________________________________________________
dropout_2 (Dropout)          (16, 64, 64)              0         
_________________________________________________________________
lstm_3 (LSTM)        

In [46]:
from tqdm import tqdm_notebook

n_batches = len(list(range(0, len(text) // SIMULTANEOUS_BATCHES - CHARS_PER_BATCH, CHARS_PER_BATCH)))

for epoch in range(0, 100):
    for x, y in tqdm_notebook(batches(text), total=n_batches, desc=f'Epoch {epoch}'):
        losses = []
        accs = []
        loss, acc = model.train_on_batch(x, y)
        losses.append(loss)
        accs.append(acc)
    print(f"Epoch: {epoch}, losses {np.mean(losses)}, accuracy {np.mean(accs)}")

Epoch: 0, losses 1.4618688821792603, accuracy 0.560546875


Epoch: 1, losses 1.485126256942749, accuracy 0.56640625


Epoch: 2, losses 1.4572129249572754, accuracy 0.5517578125


Epoch: 3, losses 1.4663128852844238, accuracy 0.5498046875


Epoch: 4, losses 1.4179461002349854, accuracy 0.5859375


...

Epoch: 95, losses 1.4481927156448364, accuracy 0.5703125


Epoch: 96, losses 1.4628201723098755, accuracy 0.5595703125


Epoch: 97, losses 1.4198832511901855, accuracy 0.5693359375


Epoch: 98, losses 1.4525489807128906, accuracy 0.5576171875


Epoch: 99, losses 1.4368534088134766, accuracy 0.5771484375


In [47]:
model.save_weights("model.h5")

In [48]:
def build_sample_model(vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SZ, batch_input_shape=(1, 1)))
    for i in range(3):
        model.add(LSTM(LSTM_SZ, return_sequences=(i != 2), stateful=True))
        model.add(Dropout(0.2))

    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    return model

pred_model = build_sample_model(len(alphabet))

In [45]:
pred_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (1, 1, 128)               9088      
_________________________________________________________________
lstm_25 (LSTM)               (1, 1, 64)                49408     
_________________________________________________________________
dropout_25 (Dropout)         (1, 1, 64)                0         
_________________________________________________________________
lstm_26 (LSTM)               (1, 1, 64)                33024     
_________________________________________________________________
dropout_26 (Dropout)         (1, 1, 64)                0         
_________________________________________________________________
lstm_27 (LSTM)               (1, 64)                   33024     
_________________________________________________________________
dropout_27 (Dropout)         (1, 64)                   0         
__________

In [60]:
pred_model.load_weights("model.h5")
inp = np.zeros((1,1))
inp[0,0] = np.random.randint(len(alphabet))
predicted = []

for i in "The":
    inp[0,0] = alphabet.index(i)
    pred_model.predict_on_batch(inp)
    predicted.append(alphabet.index(i))
    
inp[0,0] = alphabet.index(' ')
predicted.append(alphabet.index(' '))

for i in range(80):
    probs = pred_model.predict_on_batch(inp).ravel()
    pred = np.random.choice(list(range(len(alphabet))), p=probs)
    predicted.append(pred)
    inp[0,0] = pred
    
''.join([alphabet[x] for x in predicted])

'The laved bink younsfor\ntick, and was firsing?’\n\n‘It I’ve knew I do it CITE!\nE garde'